## Time Series Forecasting with CNN and Grad-CAM

In [ ]:
import numpy as np
import torch
import pandas as pd
import os
from datetime import datetime
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchvision.datasets.utils import download_and_extract_archive
from torch import nn
from torch.utils.data import Dataset, DataLoader
import shap

colormap = ListedColormap(["#ff595e","#ffca3a","#8ac926","#52a675","#1982c4","#6a4c93"], name="Custom")

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colormap.colors)
plt.rcParams['axes.axisbelow'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False


In [ ]:
csv_filename = "jena_climate_2009_2016.csv"
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
download_and_extract_archive(url=uri, download_root=os.getcwd(), filename=csv_filename+".zip")

df = pd.read_csv(csv_filename)
# convert 'Date Time' to datetime object
df['Date Time'] = df['Date Time'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y %H:%M:%S"))

# convert angle to cos & sin
df["cos(wd)"] = df["wd (deg)"].apply(lambda x: np.cos(x / 360 * 2 * np.pi))
df["sin(wd)"] = df["wd (deg)"].apply(lambda x: np.sin(x / 360 * 2 * np.pi))

df = df.drop(["wd (deg)"], axis=1)

features = df.columns.values[1:]


display(df)

In [ ]:
# check for outliers

remove_outliers = True
plot_on_timeseries = True

if plot_on_timeseries:
    fig, ax = plt.subplots(figsize=(16, 3))

for i, X in enumerate(features):
    q05 = df[X].quantile(0.05)
    q95 = df[X].quantile(0.95)
    iqr = df[X].quantile(0.75) - df[X].quantile(0.25)
    
    lower_bound = q05 - 3 * iqr
    upper_bound = q95 + 3 * iqr
    
    outliers = ((df[X] < lower_bound) | (df[X] > upper_bound))

    if outliers.sum() > 0:
        print("%s has %i outliers (lower=%.4g, upper=%.4g)" % (X, outliers.sum(), lower_bound, upper_bound))
    
    if plot_on_timeseries:
        label = X
        for obs in df.loc[outliers, "Date Time"]:
            ax.axvline(x=obs, color=colormap(i), label=label)
            label=None
    
    if remove_outliers:
        df = df[~outliers]
        

if plot_on_timeseries:
    ax.set_title("Outliers")
    ax.legend(bbox_to_anchor=(0.5, 1.2), loc='center', ncol=10)

In [ ]:
correlation_matrix = df[features].corr()
redundant_features = []
for i in range(len(features)):
    for j in range(i+1, len(features)):
        col_i = features[i]
        col_j = features[j]
        
        if correlation_matrix.loc[col_i, col_j] > 0.98:
            print("%s and %s are redundant" % (col_i, col_j))
            redundant_features.append(col_j)

df = df.drop(redundant_features, axis=1, errors='ignore')

df = df.set_index("Date Time").resample("H").mean().reset_index()

features = df.columns.values[1:]

## Visualization

In [ ]:
plot_cols = 4
plot_rows = len(features) // plot_cols + min(len(features) % plot_cols, 1)

fig, axes = plt.subplots(figsize=(16, 3 * plot_rows), nrows=plot_rows, ncols=plot_cols)

for ax, X in zip(axes.flatten(), features):
    ax.set_title(X)
    ax.hist(df[X], bins=50)
    ax.set_yticks([])

fig.tight_layout()

In [ ]:
plot_cols = 2
plot_rows = len(features) // plot_cols + min(len(features) % plot_cols, 1)

fig, axes = plt.subplots(figsize=(16, 3 * plot_rows), nrows=plot_rows, ncols=plot_cols)

sample_low = 0
sample_high = 52560 * 2 # 2 years
sample_high = -1
skip = 6

for ax, X in zip(axes.flatten(), features):
    ax.set_title(X)
    plot_x = df["Date Time"].iloc[sample_low:sample_high:skip]
    plot_y = df[X].iloc[sample_low:sample_high:skip]
    ax.plot(plot_x, plot_y, linewidth=1)

fig.tight_layout()

In [ ]:
correlation_matrix = df.corr()

for i in range(len(features)):
    for j in range(len(features)):
        if i<=j:
            correlation_matrix.iloc[i, j] = np.nan

plt.figure(figsize=(8, 7))
plt.imshow(correlation_matrix, cmap="bwr", vmin=-1, vmax=1)
plt.colorbar(label="Correlation coefficient", location="top", shrink=0.6)
plt.xticks(ticks=range(len(features)), labels=features, rotation=90)
plt.yticks(ticks=range(len(features)), labels=features, rotation=0)
plt.xlim(-0.5, len(features) - 1.5 )
plt.ylim(len(features)-0.5, 0.5 )
plt.tight_layout()
plt.show()

## Split, scale & transform

In [ ]:
hours_future = 12
hours_past = 48

time_array = df['Date Time']
X = df[features]
y = df['T (degC)'].shift(-(hours_future + hours_past))

# split into test and train datasets
time_train, time_test, X_train, X_test, y_train, y_test = train_test_split(time_array, X, y, test_size=0.3, shuffle=False)

# scale datasets
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# switch from pandas Series to numpy array
y_train = y_train.values
y_test = y_test.values

#### Prepare data for PyTorch

In [ ]:
def prepare_sequences(X, sequence_length=48):
    X_seq = []
    for i in range(sequence_length):
        X_roll = X.roll(-i, dims=0)
        X_seq.append(X_roll)
    
    X_seq = torch.stack(X_seq, dim=2)
    
    # expand the dimensions
    X_seq = X_seq[:, None, :, :]
    
    return X_seq


def prepare_data(X, y, time_array, hours_past=48, hours_future=12, batch_size=32):
    X = torch.Tensor(X)
    y = torch.Tensor(y)
    
    X_ts = prepare_sequences(X, hours_past)
    X_ts = X_ts[:-(hours_past + hours_future)]
    y_ts = y[:-(hours_past + hours_future)]
        
    time_array = time_array[:-(hours_past + hours_future)]
    
    # we verify which observations are valid i.e. there is proper time difference between the observations
    # valid_observations = ((time_array.diff(-hours_past).dt.total_seconds() / 3600 ) ==  -hours_past ) & ((time_array.diff(hours_future).dt.total_seconds() / 3600 ) ==  hours_future )
    
    x_nan_observations = torch.isnan(X_ts).any(dim=1).any(dim=1).any(dim=1)
    y_nan_observations = torch.isnan(y_ts)
    
    nan_observations = x_nan_observations | y_nan_observations
    
    time_ts = time_array[~nan_observations.numpy()]
    X_ts = X_ts[~nan_observations]
    y_ts = y_ts[~nan_observations]
    
    return time_ts, X_ts, y_ts

time_ts_train, X_ts_train, y_ts_train = prepare_data(X_train, y_train, time_train)
time_ts_test, X_ts_test, y_ts_test = prepare_data(X_test, y_test, time_test)

In [ ]:
class WeatherDataset(Dataset):
    def __init__(self,features, target):
        self.features = features
        self.target = target
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,idx):
        item = self.features[idx]
        label = self.target[idx]
        
        return item,label
    
train = WeatherDataset(X_ts_train, y_ts_train)
test = WeatherDataset(X_ts_test, y_ts_test)

train_loader = DataLoader(train, batch_size=64, shuffle=False)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

## Build model and train

In [ ]:
class CNN_GradCAM(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
                
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1, 3))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1, 3))
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(1, 2))
        
        flat_size = int((input_shape[-1] - 4 * 2) / 4 * 32 * input_shape[-2])
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(flat_size, 50)
        self.fc2 = nn.Linear(50, 1)
        
        # placeholder for the gradients
        self.gradients = None
    
    def forward(self, x):
        # 1st convolution
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # 2nd convolution
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # flatten & fully connected layers
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x
    
    def forward_cam(self, x):
        # 1st convolution
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # 2nd convolution
        x = self.conv2(x)
        x = self.relu(x)
        h = x.register_hook(self.activations_hook)
        x = self.pool(x)
        
        # flatten & fully connected layers
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x
        
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    
    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
    
    # method for the activation exctraction
    def get_activations(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        return x
    
device = torch.device("cpu")
model = CNN_GradCAM(X_ts_train.shape).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_function = nn.MSELoss()

In [ ]:
def fit(model, data_loader):
    running_loss = .0
    model.train()
    
    for idx, (inputs, labels) in tqdm(enumerate(data_loader), total=data_loader.__len__(), disable=True):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())[:, 0]
        
        loss = loss_function(preds ,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(data_loader)
    train_loss = train_loss.detach().numpy()
    return train_loss

def validate(model, data_loader):
    running_loss = .0
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())[:, 0]
            loss = loss_function(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(data_loader)
        valid_loss = valid_loss.detach().numpy()
        
        return valid_loss

train_losses = []
valid_losses = []

epochs = 10
print("Started learning for %i epochs..." % epochs)
for epoch in range(epochs):
    start = time.time()
    train_loss = fit(model, train_loader)
    train_losses.append(train_loss)
    valid_loss = validate(model, test_loader)
    valid_losses.append(valid_loss)
    time_elapsed = time.time() - start
    
    print('Epochs %i/%i (%.3g seconds)\n    Train loss = %.3g \n    Valid loss = %.3g' % (epoch+1, epochs, time_elapsed, train_loss, valid_loss))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
epochs_passed = len(train_losses)
ax.plot(range(1, epochs_passed + 1), train_losses, label="Train", linewidth=2, marker='o', markersize=4)
ax.plot(range(1, epochs_passed + 1), valid_losses, label="Validation", linewidth=2, marker='o', markersize=4)
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_xlim(1, len(train_losses))
ax.legend()
ax.grid(axis='y', alpha=0.5)
fig.tight_layout()

## Predict and evaluate

In [ ]:
def r2_score(y, y_pred):
    ss_tot = torch.sum((y - torch.mean(y)) ** 2)
    ss_res = torch.sum((y - y_pred) ** 2)
    return 1 - ss_res / ss_tot

time_ts_test_sample = time_ts_test[::5]
X_ts_test_sample = X_ts_test[::5]
y_ts_test_sample = y_ts_test[::5]

y_test_pred = model.forward(X_ts_test_sample).detach().view(-1)

print("R2 score: %.3f" % r2_score(y_ts_test_sample, y_test_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(y_ts_test_sample, y_test_pred, s=5, alpha=0.3)
ax.plot([-15, 35], [-15, 35], linewidth=1, color="black", linestyle="--")
ax.set_xlim(-15, 35)
ax.set_ylim(-15, 35)
ax.set_xlabel("Target values")
ax.set_ylabel("Predicted values")
fig.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(time_ts_test_sample, y_ts_test_sample, label="Target")
ax.plot(time_ts_test_sample, y_test_pred, label="Prediction", alpha=1, linewidth=1.5)
ax.set_ylabel("Temperature [C]")
ax.legend()
ax.set_xlim(time_ts_test_sample.values[-1000], time_ts_test_sample.values[-700])
ax.set_yticks(np.arange(-15, 35.1, 5))
ax.grid(axis='y')
fig.tight_layout()

### Grad-CAM

In [ ]:
# references
# https://github.com/jacobgil/pytorch-grad-cam/issues/233
# https://medium.com/@stepanulyanin/implementing-grad-cam-in-pytorch-ea0937c31e82
# https://arxiv.org/pdf/2001.07582.pdf

In [ ]:
def reshape_transform(tensor, target_size):
    """ 
    Transforms a tensor to the required shape by interpolation
    Used to transform the tensor before last pooling layer to input size.
    Note: this should only interpolate time axis, not the feature axis!
    """
    tensor = tensor.reshape((1, 1, tensor.shape[0], tensor.shape[1]))
    image_with_single_row = tensor[:, None, :, :]
    # Lets make the time series into an image with 16 rows for easier visualization on screen later
    return torch.nn.functional.interpolate(tensor, target_size, mode='bilinear')

def genereate_gradcam_explanation(model, x):
    pred = model.forward_cam(x)
    pred.backward()
    gradients = model.get_activations_gradient()
    print("Gradients shape: {}".format(gradients.shape))
    
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    
    print("Pooled Gradients shape: {}".format(pooled_gradients.shape))
    
    activations = model.get_activations(x).detach()
    print("Activations shape: {}".format(activations.shape))

    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]

    mean_activations = torch.mean(activations, dim=1).squeeze()
    mean_activations = torch.maximum(mean_activations, torch.tensor(0))
    mean_activations = reshape_transform(mean_activations, (x.shape[-2], x.shape[-1]))
    return mean_activations


def plot_explanation_heatmap(heatmap, std_vmin=False, use_abs=False):
    """
    Function to plot explanation heatmap
    """
    if use_abs:
        heatmap = abs(heatmap)
    
    vmax = heatmap.mean() + 3 * heatmap.std()
    if std_vmin:
        vmin = heatmap.mean() - 3 * heatmap().std()
    else:
        vmin=0
        
    t = heatmap.shape[-1] -1 # time 
    
    fig, ax =plt.subplots(figsize=(8, 4))
    ax.imshow(heatmap.squeeze(), aspect=2, vmin=vmin, vmax=vmax)
    ax.set_xlabel("Time back [h]")
    ax.set_yticks(range(12))
    ax.set_yticklabels(features)
    ax.set_xticks(np.linspace(0, t, 8).astype(int))
    ax.set_xticklabels(np.linspace(t, 0, 8).astype(int))
    fig.tight_layout()

In [ ]:
# batch of data for explanations
x_batch, y_batch = next(iter(test_loader))

In [ ]:
i = 30 # select one observation from batch
x = x_batch[i:i+1]
y = y_batch[i:i+1]

explanation = genereate_gradcam_explanation(model, x)
plot_explanation_heatmap(explanation)